# Lab 15.2: Commercial Insurance Integration

**Duration:** 12 minutes  
**Difficulty:** Advanced  

## Learning Objectives

In this notebook, you will:
- Create business customer profiles with industry classifications
- Build commercial insurance products (General Liability, Workers Comp, Cyber)
- Implement multi-policy commercial accounts
- Query complex commercial insurance relationships

---

## Step 1: Connect to Neo4j

Establish connection and verify the database includes life insurance from the previous notebook.

In [ ]:
from neo4j import GraphDatabase
import pandas as pd
from datetime import datetime, date
import uuid

# Connect to Neo4j Enterprise instance
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "password"))

def run_query(query, parameters=None):
    with driver.session(database="insurance") as session:
        result = session.run(query, parameters)
        return [record.data() for record in result]

# Verify current database state
current_state = run_query("""
MATCH (n) 
RETURN labels(n)[0] AS node_type, count(n) AS count
ORDER BY count DESC
""")

print("Current Database State:")
for record in current_state:
    print(f"  {record['node_type']}: {record['count']} nodes")

## Step 2: Create Commercial Insurance Products

### Business Context: Commercial Insurance Lines

**General Liability Insurance:**
- Covers third-party bodily injury and property damage
- Products and completed operations coverage
- Personal and advertising injury protection
- Medical payments to others
- Occurrence or claims-made basis
- Per occurrence and aggregate limits

**Workers Compensation Insurance:**
- Mandatory in most states for businesses with employees
- Covers employee injuries and illnesses arising from employment
- Medical expenses, lost wages, rehabilitation, death benefits
- Rated by payroll and employee classification codes
- Experience modification factor adjusts pricing
- Employers liability coverage included

**Cyber Liability Insurance:**
- First-party coverage: Business interruption, data restoration, cyber extortion
- Third-party coverage: Privacy liability, network security liability, media liability
- Breach response services: Legal, forensics, notification, credit monitoring
- Regulatory fines and penalties coverage
- Claims-made basis with retroactive dates
- Rapid evolution due to emerging threats

In [ ]:
# Create commercial insurance products
commercial_products_query = """
// Create Commercial Insurance Products
CREATE (general_liability:Product:Insurance:Commercial {
  id: randomUUID(),
  product_id: "PROD-COMM-001",
  product_name: "Business General Liability",
  product_type: "Commercial",
  insurance_line: "General Liability",
  coverage_type: "Occurrence",
  
  // Coverage details
  min_coverage: 500000.00,
  max_coverage: 10000000.00,
  aggregate_limit: 2000000.00,
  deductible_options: [1000, 2500, 5000, 10000],
  
  // Commercial specific
  industry_codes: ["541511", "541512", "541519"], // Computer Systems Design
  territory: "Texas",
  policy_term: 12, // months
  
  created_at: datetime(),
  created_by: "commercial_products",
  version: 1
})

CREATE (workers_comp:Product:Insurance:Commercial {
  id: randomUUID(),
  product_id: "PROD-COMM-002",
  product_name: "Workers Compensation",
  product_type: "Commercial",
  insurance_line: "Workers Compensation",
  coverage_type: "Statutory",
  
  // Coverage details
  experience_modification: 1.0,
  classification_codes: ["8810", "8820"], // Clerical, Software Development
  payroll_basis: "Per $100 of payroll",
  
  // State specific
  state_jurisdiction: "Texas",
  twcc_approved: true,
  return_to_work: true,
  
  created_at: datetime(),
  created_by: "commercial_products",
  version: 1
})

CREATE (cyber_insurance:Product:Insurance:Commercial {
  id: randomUUID(),
  product_id: "PROD-COMM-003",
  product_name: "CyberGuard Pro",
  product_type: "Commercial",
  insurance_line: "Cyber Liability",
  coverage_type: "Claims Made",
  
  // Cyber specific coverage
  data_breach_coverage: 5000000.00,
  business_interruption: 2000000.00,
  cyber_extortion: 1000000.00,
  regulatory_fines: 1000000.00,
  
  // Features
  breach_response_services: true,
  forensic_investigation: true,
  credit_monitoring: true,
  pr_crisis_management: true,
  
  created_at: datetime(),
  created_by: "cyber_products",
  version: 1
})

RETURN general_liability.product_name AS gl_product,
       workers_comp.product_name AS wc_product,
       cyber_insurance.product_name AS cyber_product
"""

product_results = run_query(commercial_products_query)
print("Commercial Insurance Products Created:")
for record in product_results:
    print(f"  General Liability: {record['gl_product']}")
    print(f"  Workers Compensation: {record['wc_product']}")
    print(f"  Cyber Liability: {record['cyber_product']}")

## Step 3: Create Business Customer Profiles

### Business Context: Commercial Underwriting

**Key Business Characteristics:**
- **NAICS Code**: North American Industry Classification System (6-digit industry code)
- **SIC Code**: Standard Industrial Classification (legacy 4-digit code)
- **Years in Business**: New ventures higher risk than established businesses
- **Annual Revenue**: Indicates business size and exposure
- **Employee Count**: Drives workers comp and liability pricing
- **Payroll**: Basis for workers compensation premium calculation

**Risk Assessment Factors:**
- Previous claims history and loss experience
- Safety programs and loss control measures
- Security certifications (SOC 2, ISO standards)
- Professional certifications and training
- Business operations and service descriptions
- Geographic territory and exposure areas

In [ ]:
# Create business customer profiles
business_customer_query = """
// Create Business Customers
CREATE (tech_corp:Customer:Business {
  id: randomUUID(),
  customer_id: "CUST-BUS-001",
  business_name: "Innovative Tech Solutions LLC",
  business_type: "LLC",
  dba_name: "TechSolutions",
  
  // Business details
  industry: "Computer Systems Design",
  naics_code: "541511",
  sic_code: "7371",
  years_in_business: 8,
  
  // Contact information
  business_phone: "214-555-0250",
  business_email: "info@techsolutions.com",
  website: "www.techsolutions.com",
  
  // Address
  street_address: "1500 Technology Drive, Suite 200",
  city: "Dallas",
  state: "TX",
  postal_code: "75201",
  country: "USA",
  
  // Financial information
  annual_revenue: 12500000.00,
  employee_count: 85,
  payroll: 6800000.00,
  
  // Risk factors
  data_classification: "Confidential/PII",
  security_certifications: ["SOC 2", "ISO 27001"],
  previous_claims: 0,
  
  created_at: datetime(),
  created_by: "commercial_underwriting",
  version: 1
})

CREATE (consulting_firm:Customer:Business {
  id: randomUUID(),
  customer_id: "CUST-BUS-002",
  business_name: "Strategic Business Advisors Inc",
  business_type: "Corporation",
  dba_name: "SBA Consulting",
  
  // Business details
  industry: "Management Consulting",
  naics_code: "541611",
  sic_code: "8742",
  years_in_business: 15,
  
  // Contact information
  business_phone: "972-555-0290",
  business_email: "contact@sbaconsulting.com",
  website: "www.sbaconsulting.com",
  
  // Address
  street_address: "8900 Business Center Drive",
  city: "Plano",
  state: "TX",
  postal_code: "75024",
  country: "USA",
  
  // Financial information
  annual_revenue: 5800000.00,
  employee_count: 42,
  payroll: 3200000.00,
  
  // Risk factors
  professional_services: true,
  errors_omissions_exposure: "High",
  client_contracts: "Fortune 500",
  
  created_at: datetime(),
  created_by: "commercial_underwriting",
  version: 1
})

RETURN tech_corp.business_name AS tech_business,
       tech_corp.employee_count AS tech_employees,
       consulting_firm.business_name AS consulting_business,
       consulting_firm.employee_count AS consulting_employees
"""

business_results = run_query(business_customer_query)
print("\nBusiness Customers Created:")
for record in business_results:
    print(f"  {record['tech_business']} - {record['tech_employees']} employees")
    print(f"  {record['consulting_business']} - {record['consulting_employees']} employees")

## Step 4: Create Commercial Insurance Policies

### Business Context: Commercial Package Policies

**Multi-Policy Commercial Accounts:**
- Businesses typically carry multiple coverage types
- Package discounts for bundled policies (5-15% savings)
- Coordinated policy periods for renewal efficiency
- Shared limits and deductibles across coverages

**General Liability Coverage Triggers:**
- **Occurrence Basis**: Covers incidents during policy period regardless of when claim filed
- **Claims Made Basis**: Covers claims filed during policy period for covered incidents

**Workers Compensation Rating:**
- Base rate per $100 of payroll by employee classification
- Experience modification factor adjusts for claim history
- Schedule credits for safety programs, size, and loss control

In [ ]:
# Create commercial policies for tech company
tech_policies_query = """
// Match business customer and products
MATCH (tech_corp:Customer:Business {customer_id: "CUST-BUS-001"})
MATCH (gl_product:Product {product_id: "PROD-COMM-001"})
MATCH (wc_product:Product {product_id: "PROD-COMM-002"})
MATCH (cyber_product:Product {product_id: "PROD-COMM-003"})

// Create General Liability Policy
CREATE (gl_policy:Policy:Commercial:Active {
  id: randomUUID(),
  policy_number: "POL-GL-001500",
  product_type: "Commercial",
  business_type: "General Liability",
  
  // Coverage details
  coverage_limit: 2000000.00,
  aggregate_limit: 4000000.00,
  deductible: 2500.00,
  annual_premium: 3850.00,
  
  // Policy dates
  effective_date: date("2024-06-01"),
  expiration_date: date("2025-06-01"),
  
  // Commercial specific
  coverage_territory: "Texas",
  industry_code: "541511",
  employee_coverage: 85,
  payroll: 6800000.00,
  gross_receipts: 12500000.00,
  
  // Property locations
  property_locations: ["1500 Technology Drive, Dallas, TX"],
  building_construction: "Steel Frame - Fire Resistive",
  security_features: ["24/7 Security", "Access Controls", "CCTV"],
  
  // Status
  policy_status: "Active",
  premium_status: "Current",
  last_payment_date: date("2024-06-01"),
  next_payment_due: date("2024-12-01"),
  
  created_at: datetime(),
  created_by: "commercial_underwriting",
  version: 1
})

// Create Workers Compensation Policy
CREATE (wc_policy:Policy:Commercial:Active {
  id: randomUUID(),
  policy_number: "POL-WC-001500",
  product_type: "Commercial",
  business_type: "Workers Compensation",
  
  // Coverage details
  coverage_limit: 1000000.00,
  deductible: 0.00,
  annual_premium: 18750.00,
  
  // Policy dates
  effective_date: date("2024-06-01"),
  expiration_date: date("2025-06-01"),
  
  // Workers comp specific
  payroll: 6800000.00,
  employee_count: 85,
  experience_mod: 0.95,
  classification_codes: ["8810 - Clerical: $2,400,000", "8820 - Software Dev: $4,400,000"],
  rates: ["8810: $0.35 per $100", "8820: $0.28 per $100"],
  
  // State compliance
  state_jurisdiction: "Texas",
  twcc_coverage: "Yes",
  return_to_work_program: true,
  
  // Status
  policy_status: "Active",
  premium_status: "Current",
  last_payment_date: date("2024-06-01"),
  next_payment_due: date("2024-12-01"),
  
  created_at: datetime(),
  created_by: "commercial_underwriting",
  version: 1
})

// Create Cyber Liability Policy
CREATE (cyber_policy:Policy:Commercial:Active {
  id: randomUUID(),
  policy_number: "POL-CYB-001500",
  product_type: "Commercial",
  business_type: "Cyber Liability",
  
  // Coverage details
  coverage_limit: 5000000.00,
  deductible: 10000.00,
  annual_premium: 12500.00,
  
  // Policy dates
  effective_date: date("2024-06-01"),
  expiration_date: date("2025-06-01"),
  
  // Cyber specific coverage
  data_breach_limit: 5000000.00,
  business_interruption_limit: 2000000.00,
  cyber_extortion_limit: 1000000.00,
  regulatory_fines_limit: 1000000.00,
  
  // Risk profile
  data_records: 125000,
  pii_records: 85000,
  pci_compliance: true,
  security_training: "Annual",
  incident_response_plan: true,
  
  // Features included
  breach_coach: "24/7 Hotline",
  forensic_services: "Included",
  credit_monitoring: "2 Years",
  pr_services: "Included",
  
  // Status
  policy_status: "Active",
  premium_status: "Current",
  last_payment_date: date("2024-06-01"),
  next_payment_due: date("2024-12-01"),
  
  created_at: datetime(),
  created_by: "cyber_underwriting",
  version: 1
})

// Create Relationships
CREATE (tech_corp)-[:HOLDS_POLICY {
  start_date: gl_policy.effective_date,
  policy_role: "Named Insured"
}]->(gl_policy)

CREATE (tech_corp)-[:HOLDS_POLICY {
  start_date: wc_policy.effective_date,
  policy_role: "Named Insured"
}]->(wc_policy)

CREATE (tech_corp)-[:HOLDS_POLICY {
  start_date: cyber_policy.effective_date,
  policy_role: "Named Insured"
}]->(cyber_policy)

CREATE (gl_policy)-[:BASED_ON {
  underwriting_date: gl_policy.effective_date,
  risk_assessment: "Approved - Technology Sector"
}]->(gl_product)

CREATE (wc_policy)-[:BASED_ON {
  underwriting_date: wc_policy.effective_date,
  risk_assessment: "Approved - Experience Credit"
}]->(wc_product)

CREATE (cyber_policy)-[:BASED_ON {
  underwriting_date: cyber_policy.effective_date,
  risk_assessment: "Approved - High Security Standards"
}]->(cyber_product)

RETURN tech_corp.business_name AS business,
       gl_policy.policy_number AS general_liability,
       wc_policy.policy_number AS workers_comp,
       cyber_policy.policy_number AS cyber
"""

policy_results = run_query(tech_policies_query)
print("\nCommercial Insurance Policies Created:")
for record in policy_results:
    print(f"  Business: {record['business']}")
    print(f"  General Liability: {record['general_liability']}")
    print(f"  Workers Compensation: {record['workers_comp']}")
    print(f"  Cyber Liability: {record['cyber']}")

## Step 5: Commercial Insurance Verification Queries

Analyze the commercial insurance portfolio with comprehensive queries.

In [ ]:
# Query 1: Complete commercial account overview
account_query = """
MATCH (b:Customer:Business)-[:HOLDS_POLICY]->(p:Policy:Commercial)
WITH b,
     count(p) AS policy_count,
     sum(p.annual_premium) AS total_premium,
     sum(p.coverage_limit) AS total_coverage,
     collect(p.business_type) AS coverage_types
RETURN b.business_name AS business,
       b.industry AS industry,
       b.employee_count AS employees,
       b.annual_revenue AS revenue,
       policy_count,
       total_premium,
       total_coverage,
       coverage_types
ORDER BY total_premium DESC
"""

account_df = pd.DataFrame(run_query(account_query))
print("\nCommercial Account Overview:")
print(account_df.to_string(index=False))

In [ ]:
# Query 2: Workers compensation analysis
wc_analysis_query = """
MATCH (b:Customer:Business)-[:HOLDS_POLICY]->(p:Policy:Commercial)
WHERE p.business_type = "Workers Compensation"
RETURN b.business_name AS business,
       p.policy_number AS policy,
       p.employee_count AS employees,
       p.payroll AS annual_payroll,
       p.experience_mod AS experience_mod,
       p.annual_premium AS premium,
       round(p.annual_premium * 100.0 / p.payroll, 4) AS rate_per_100_payroll
ORDER BY p.payroll DESC
"""

wc_df = pd.DataFrame(run_query(wc_analysis_query))
print("\nWorkers Compensation Analysis:")
print(wc_df.to_string(index=False))

In [ ]:
# Query 3: Cyber liability risk profile
cyber_risk_query = """
MATCH (b:Customer:Business)-[:HOLDS_POLICY]->(p:Policy:Commercial)
WHERE p.business_type = "Cyber Liability"
RETURN b.business_name AS business,
       p.policy_number AS policy,
       p.data_records AS total_records,
       p.pii_records AS pii_records,
       p.data_breach_limit AS breach_coverage,
       p.business_interruption_limit AS bi_coverage,
       p.annual_premium AS premium,
       p.pci_compliance AS pci_compliant,
       p.incident_response_plan AS has_ir_plan
"""

cyber_df = pd.DataFrame(run_query(cyber_risk_query))
print("\nCyber Liability Risk Profile:")
print(cyber_df.to_string(index=False))

In [ ]:
# Query 4: Commercial product performance
product_performance_query = """
MATCH (prod:Product:Commercial)<-[:BASED_ON]-(p:Policy:Commercial)
WITH prod,
     count(p) AS policies_written,
     sum(p.annual_premium) AS total_premium,
     avg(p.annual_premium) AS avg_premium,
     sum(p.coverage_limit) AS total_coverage
RETURN prod.product_name AS product,
       prod.insurance_line AS line,
       policies_written,
       round(total_premium, 2) AS total_premium,
       round(avg_premium, 2) AS avg_premium,
       total_coverage
ORDER BY total_premium DESC
"""

product_df = pd.DataFrame(run_query(product_performance_query))
print("\nCommercial Product Performance:")
print(product_df.to_string(index=False))

In [ ]:
# Query 5: Multi-policy account analysis
multi_policy_query = """
MATCH (b:Customer:Business)-[:HOLDS_POLICY]->(p:Policy:Commercial)
WITH b,
     collect({
       type: p.business_type,
       policy: p.policy_number,
       coverage: p.coverage_limit,
       premium: p.annual_premium
     }) AS policies,
     sum(p.annual_premium) AS account_premium
WHERE size(policies) > 1
RETURN b.business_name AS business,
       size(policies) AS policy_count,
       account_premium,
       policies
ORDER BY account_premium DESC
"""

multi_policy_results = run_query(multi_policy_query)
print("\nMulti-Policy Commercial Accounts:")
for record in multi_policy_results:
    print(f"\n  Business: {record['business']}")
    print(f"  Total Policies: {record['policy_count']}")
    print(f"  Account Premium: ${record['account_premium']:,.2f}")
    print("  Policy Details:")
    for policy in record['policies']:
        print(f"    - {policy['type']}: {policy['policy']} (${policy['premium']:,.2f})")

## Step 6: Summary

### What You've Accomplished:

- Created **General Liability Insurance** products for commercial operations
- Created **Workers Compensation Insurance** with payroll-based rating
- Created **Cyber Liability Insurance** with comprehensive breach response coverage
- Built business customer profiles with NAICS codes and industry classifications
- Implemented multi-policy commercial accounts with package coverage
- Established risk profiles including security certifications and loss history

### Key Commercial Insurance Concepts:

1. **Multi-Line Commercial Coverage**: Businesses require diverse insurance products working together
2. **Industry-Based Underwriting**: NAICS/SIC codes drive risk classification and pricing
3. **Payroll Rating Basis**: Workers comp premiums calculated per $100 of payroll by job classification
4. **Experience Modification**: Historical loss experience adjusts pricing (0.95 = 5% credit)
5. **Cyber Risk Management**: Modern businesses face significant technology-related exposures

---

**Next:** Continue to notebook 03 for Specialty Insurance Products.